In [20]:
from yfinance import *
import pandas as pd
import sqlite3

# Tickers for our stokcs
txn = Ticker("TXN")
regn = Ticker("REGN")
amd = Ticker("AMD")
gral = Ticker("GRAL")
klac = Ticker("KLAC")
market = Ticker("^GSPC")


# Historical data YTD
txn_ytd = txn.history(period="ytd")
market_ytd = market.history(period="ytd")

# Data including date & % up/down for each day
market_trend = ((market_ytd['Close'] - market_ytd['Open']) / market_ytd['Open']) * 100

txn_trend = ((txn_ytd['Close'] - txn_ytd['Open']) /
             txn_ytd['Open']) * 100


# These loops mark True for a positive shift (Stock went up) and False for a negative shift (Market went down)
market_arr = []
for tr in market_trend:
    tr = tr > 0
    market_arr.append(tr)


txn_arr = []
for tr in txn_trend:
    tr = tr > 0
    txn_arr.append(tr)



# With %Yield
# market_data = {"Date": market_ytd.index, "Trend": ((market_ytd['Close'] - market_ytd['Open']) / market_ytd['Open']) * 100}

# With Boolean markers
market_data = {"Date": market_ytd.index, "Trend": market_arr}

txn_data = {"Date": txn_ytd.index, "Trend": txn_arr}

# Dataframes for Market & TXN
txn_df = pd.DataFrame(txn_data)
market_df = pd.DataFrame(market_data)

# Local Memory (SQLITE)
conn = sqlite3.connect(':memory:')

# Importing Dataframes into SQL
txn_df.to_sql('txn_data', conn, index=False)
market_df.to_sql('market_data', conn, index=False)

# Querying to find when TXN went up when the Market was down
query = """
SELECT Date, Trend
FROM txn_data
WHERE (Trend = TRUE
    AND Date IN(
        SELECT Date
        FROM market_data
        WHERE Trend = FALSE));
"""

result = pd.read_sql_query(query, conn)
print(result)



# CSV Files
# txn_df.to_csv('txn_trend.csv', index=False)
# market_df.to_csv('market_trend.csv', index=False)





                         Date  Trend
0   2025-01-06 00:00:00-05:00      1
1   2025-01-14 00:00:00-05:00      1
2   2025-01-31 00:00:00-05:00      1
3   2025-02-14 00:00:00-05:00      1
4   2025-02-20 00:00:00-05:00      1
5   2025-03-26 00:00:00-04:00      1
6   2025-04-17 00:00:00-04:00      1
7   2025-04-21 00:00:00-04:00      1
8   2025-05-09 00:00:00-04:00      1
9   2025-05-20 00:00:00-04:00      1
10  2025-05-28 00:00:00-04:00      1
11  2025-06-04 00:00:00-04:00      1
